In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from IPython.display import clear_output
import pandas as pd 

In [2]:
# pass in year as a string to get the player game logs for that season
def retrieve_game_logs(season):
    assert type(season) == str
    assert int(season)
    
    active_players = [p for p in players.get_active_players()]
    player_list = [p for p in players.get_active_players()]
    
    game_logs = []
    total = len(active_players)
    while len(player_list) > 0:
        for player in active_players:
            clear_output(wait=True)
            if player in player_list:
                name = player['full_name']
                completed = total - len(player_list)
                print(f'{round((completed/total)*100,2)}% completion')
                print(f'Retrieving {season} game logs for {name}')
                try:
                    game_log = playergamelog.PlayerGameLog(player_id=player['id'], season=season).get_data_frames()[0]
                    game_logs.append(game_log)
                    player_list.remove(player)
                except:
                    print(f'FAILED to retrieve logs for {name}, we\'ll get em next time.')
            else:
                continue
                
    print('100%')
    del active_players
    return pd.concat(game_logs)

In [ ]:
df = retrieve_game_logs('2020')

player_names = {p['id']:p['full_name'] for p in players.get_active_players()}
df.columns = [col.lower() for col in df.columns]

99.83% completion
Retrieving 2020 game logs for Lou Williams


In [ ]:
df['player_name'] = df.apply(lambda x: player_names[x['player_id']], axis=1)

In [ ]:
df = df[['season_id', 'player_id', 'player_name', 'game_id', 'game_date', 'matchup', 'wl',
       'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
       'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'plus_minus', 'video_available']]

In [34]:
df.to_csv('test_data/2020_game_logs.csv',index=False)